In [1]:
import yfinance as yf

In [2]:
symbol = "spy"
period = "1d"
interval = "5m"
file_name = f"{symbol}.csv"

In [3]:
ticker = yf.Ticker(symbol)

In [4]:
data = ticker.history(period=period, interval=interval, auto_adjust = True, prepost=True)

In [5]:
data=data.drop(columns=["Dividends", "Stock Splits"])
data['Adj Close']=data['Close']
data.tail()

,Open,High,Low,Close,Volume,Adj Close
Datetime,,,,,,
2021-08-26 09:20:00-04:00,448.560000,448.630000,448.545000,448.550000,0,448.550000
2021-08-26 09:25:00-04:00,448.600000,448.650000,448.450000,448.580000,0,448.580000
2021-08-26 09:30:00-04:00,448.609985,448.839996,448.500000,448.820007,1723412,448.820007
2021-08-26 09:35:00-04:00,448.839996,448.859985,448.350006,448.380005,777366,448.380005
2021-08-26 09:40:09-04:00,448.399994,448.399994,448.399994,448.399994,0,448.399994


In [6]:
data.to_csv(file_name)
#cleaned up in vscode to change to "Date Time" and to remove "-04:00"

In [7]:
INITIAL_BUDGET = 5000
BUDGET_USE = 0.50
RISK_PERCENT = 0

In [8]:
import datetime
import math
market_opentime = datetime.time(9,30,0) #6:30 AM PT
arb_tradestoptime = datetime.time(14,0,0) #11 AM PT

In [9]:
market_opentime

datetime.time(9, 30)

In [21]:

from pyalgotrade import strategy
from pyalgotrade.barfeed import csvfeed
from pyalgotrade.technical import macd

In [22]:
# # Load the bar feed from the CSV file
feed = csvfeed.GenericBarFeed(frequency="minute")
feed.addBarsFromCSV(symbol, file_name)

In [23]:
def safe_round(value, digits):
    if value is not None:
        value = round(value, digits)
    return value

In [24]:
class MyStrategy(strategy.BacktestingStrategy):
    def __init__(self, feed, instrument):
        super(MyStrategy, self).__init__(feed)
        self.__position = None
        #Looking for a MACD crossover
        #self.__sma = ma.SMA(feed[instrument].getCloseDataSeries(), 15)
        self.__macd = macd.MACD(feed[instrument].getCloseDataSeries(), 12, 26, 9)
        self.__instrument = instrument

    def onEnterOk(self, position):
        execInfo = position.getEntryOrder().getExecutionInfo()
        self.info(f"BUY at ${execInfo.getPrice()}")
    
    def onExitOk(self, position):
        execInfo = position.getExitOrder().getExecutionInfo()
        self.info(f"SELL at ${execInfo.getPrice()}")
        self.__position = None
    
    def onBars(self, bars):
        # Wait for enough bars to be available to calculate a SMA.
        if self.__macd.getHistogram()[-1] is None:
            return
        if self.__macd.getHistogram()[-2] is None:
            return
        
        bar = bars[self.__instrument]
        if self.__position is None:
            if market_opentime <= bar.getDateTime().time() <= arb_tradestoptime:
                if self.__macd.getHistogram()[-1]>0 and self.__macd.getHistogram()[-2]<0:
                    print(f"Crossover UP detected at {bar.getDateTime()}. {symbol.upper()} price was ${safe_round(bar.getClose(), 2)}. Buy some {math.ceil(bar.getClose())} CALLS.")
                    self.__position = self.enterLong(self.__instrument, 10, True)                   
                    
                if self.__macd.getHistogram()[-1]<0 and self.__macd.getHistogram()[-2]>0:
                    self.__position = self.enterShort(self.__instrument, 10, True)                    
                    print(f"Crossover DOWN detected at {bar.getDateTime()}. {symbol.upper()} price was ${safe_round(bar.getClose(), 2)}. Buy some {math.floor(bar.getClose())} PUTS.") 

        if self.__position:
            if self.__position.getEntryOrder().getAction() == 4:
                if self.__macd.getHistogram()[-1] > self.__macd.getHistogram()[-2]:
                    print(f"Momentum slowing at {bar.getDateTime()}.")
                    self.__position.exitMarket()
            else:
                if self.__macd.getHistogram()[-1] < self.__macd.getHistogram()[-2]:
                    print(f"Momentum slowing at {bar.getDateTime()}.")
                    self.__position.exitMarket()
                    
        
            
                
            

        #self.info("%s %s" % (bar.getClose(), safe_round(self.__macd.getHistogram()[-1], 4)))

In [25]:
ms1 = MyStrategy(feed, symbol)
ms1.run()

2021-08-26 09:35:00 strategy [INFO] BUY at $448.8399963378906
2021-08-26 09:40:09 broker.backtesting [DEBUG] Not enough volume to fill spy market order [2] for 10 share/s


Crossover UP detected at 2021-08-26 09:30:00. SPY price was $448.82. Buy some 449 CALLS.
Momentum slowing at 2021-08-26 09:35:00.
Momentum slowing at 2021-08-26 09:40:09.


Exception: Exit order is active and it should be canceled first

In [ ]:
8/25 449P Winner
9:40 0.27
9:44 0.33   
9:55 0.28

In [ ]:
8/24 448P TF Loser, Winner
8:00 0.84
8:10 0.82
8:34 1.09
9:30 1.16

448C Winner
9:55 0.84
10:20 0.89
10:05 0.93    

In [ ]:
8/23 446C
6:30 0.24

7:05 1.05

447P - LOSER
8:20 0.29
8:55 
    

In [ ]:
#reverse order:
option time price 
444P @10:01 $0.76    $0.81@10:03
444C @07:26 $0.64    $0.68@07:30
443C @09:21 $0.36    $0.82@12:27
443P @07:01 $0.24    $0.37@07:36
442P @9:40 $0.65     $0.70@09:47
443C @8:35 $0.85     $1.02@08:48
442P @7:35 $0.82     $1.24@07:54
443C @11:00 $
    
445P 6:35 0.40-0.47
446C 6:45 0.28-0.44
445P 7:00 0.30-0.55calc
446C 7:20 0.28-0.36
445P 7:25 0.24-0.34
446C 7:33 0.24
    
    8/17 445C 1.00 OPEn @ 7:25 1.20 High @ 7:29 1.13 Open @ 7:30
    8/17 443P 0.89 Open @ 8:25 1.11 High @ 8:29 1.07 Open @ 8:30
    8/17 443C 1.04 Open @ 10:25 1.25 High @ 10:29 1.17 Open @ 10:35
    
    8/18 444C @6:45 0.54O 1.13H @7:19 1.05O @ 7:25
    8/18 444P @7:50 0.77O 1.16H@8:18 0.99O@8:20   
    8/18 444C @9:15 0.44O  0.46H @9:15 0.42O@9:20
    8/18 444C @9:55 0.28O 0.34H@10:00 0.29O@10:15
    8/18 443P @10:25 0.57O 0.65H@10:27 0.56O@10:30
    8/18 444C @10:35 0.24O  0.62H@11:05 0.38O@11:10
    
    8/19 437P @7:35 1.78 2.22H@7:48 2.14O@7:50
    8/19 440C @8:20 1.85 1.86@8:25 LOSER if SL
    8/19 441C @8:45 1.39O
    
    8/20 442P @7:40 0.66O  0.88H @7:50 0.71O@8:10
    8/20 444C @9:35 0.22O  0.31H@9:45 0.30@9:50 - does not match ToS
    8/20 443P @10:00 0.41O 0.42H 0.36O @10:10 0.66H@10:55
                
        
    